# <center> Data Mining In Action
## <center> [Raiffeisen Data Cup](https://boosters.pro/champ_11)

<center> *До 23.03.2018*

Общий подход (заимствован из бейзлайна):
- Добавляем к каждой транзакции столбец: is_work (если транзакция находится в пределах 0.02 от дома клиента)
- Добавляем к каждой транзакции столбец: is_home (если транзакция находится в пределах 0.02 от работы клиента)
- Обучаем классификатор предсказывающий вероятность (is_home == 1) для транзакции
- Обучаем классификатор предсказывающий вероятность (is_work == 1) для транзакции

Точность определения местоположения:
- для классификатора is_home: ~3x%
- для классификатора is_work: ~2x%
- общая оценка на Public Leaderboard: **0.357375**

Используемые версии библиотек:

In [1]:
#!pip install watermark
%load_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,sklearn,xgboost,reverse_geocoder,requests -g

CPython 3.5.2
IPython 6.2.1

numpy 1.13.3
scipy 1.0.0
pandas 0.22.0
matplotlib 2.1.2
sklearn 0.19.1
xgboost 0.7.post3
reverse_geocoder n
requests 2.18.4

compiler   : GCC 5.4.0 20160609
system     : Linux
release    : 4.4.119-boot2docker
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   : e5336b6e8ecc4422ae6975a19ef72c16e67d0558


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import reverse_geocoder
import datetime
import re
import xgboost as xgb
import sklearn

from sklearn.model_selection import train_test_split

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = (9,6)

In [3]:
RANDOM_SEED = 7
np.random.seed(RANDOM_SEED)

### Задача
По обезличенным данным об истории транзакций и информацию о мерчантах (место, позволяющее принимать платежи с использованием банковской пластиковой карты) требуется предсказать две пары координат: дом и работу клиента.

### Описание данных

Требуется предсказать две пары координат (работы и дома): 
* (`_HOME_LAT_`, `_HOME_LON_`); 
* (`_WORK_LAT_`, `_WORK_LON_`).

|Признак|Описание|Тип|
|-------|--------|---|
|`terminal_id`|идентификационный номер терминала|**уникальный**|
|`customer_id`|идентификационный номер клиента|**уникальный**|
|`amount`|количество потраченных средств за транзакцию|**численный**|
|`country`|страна|**категориальный**|
|`city`|город|**категориальный**|
|`currency`|валюта|**категориальный**|
|`mcc`|код категории продавца|**уникальный**|
|`transaction_date`|дата транзакции|**категориальный**|
|`atm_address`|адрес банкомата|**категориальный**|
|`pos_address`|адрес установки pos-терминала|**категориальный**|
|`pos_address_lat`|адрес установки pos-терминала широта|**численный**|
|`pos_address_lon`|адрес установки pos-терминала долгота|**численный**|
|`work_add_lat`|широта работы клиента|**численный**|
|`work_add_lon`|долгота работы клиента|**численный**|
|`home_add_lat`|широта дома клиента|**численный**|
|`home_add_lon`|долгота дома клиента|**численный**|

### Оценка алгоритма
Метрикой качества в задаче является процент попаданий в окружность радиуса 0.02 градуса относительно координат дома и работы клиента. 
$$\text{Score} = \frac{N_h}{N} \times 100\% \text{, где } N \text{ - число всех клиентов.}$$

$$ N_h\displaystyle\sum_{i=1}^{N} I_i, 
I_i = \begin{cases}
    1, \quad ||\hat{r_i} - r_i|| \leq R \\
    0, \quad ||\hat{r_i} - r_i|| > R
\end{cases} $$

* $R$ - радиус круга (0.02 градуса);
* $\hat{r_i}$ - предсказанные координаты;
* $r_i$ - точные координаты;
* $r_i$ = (lat, lon).
 
$$||\hat{r_i} - r_i|| = \sqrt{(\hat{lat} - lat)^2 + (\hat{lon} - lon)^2}$$

### Полезное
* [MCC codes](https://github.com/greggles/mcc-codes)
* [Hint Geo](https://www.youtube.com/watch?v=NVKDSNM702k) - from [Tinkoff competition](https://boosters.pro/champ_3)

In [4]:
# Определим типы колонок для экономии памяти
dtypes = {
    'transaction_date': str,
    'atm_address': str,
    'country': str,
    'city': str,
    'amount': np.float32,
    'currency': np.float32,
    'mcc': str,
    'customer_id': str,
    'pos_address': str,
    'atm_address': str,
    'pos_adress_lat': np.float32,
    'pos_adress_lon': np.float32,
    'pos_address_lat': np.float32,
    'pos_address_lon': np.float32,
    'atm_address_lat': np.float32,
    'atm_address_lon': np.float32,
    'home_add_lat': np.float32,
    'home_add_lon': np.float32,
    'work_add_lat': np.float32,
    'work_add_lon': np.float32,
}

# для экономии памяти будем загружать только часть атрибутов транзакций
usecols_train = [
    'customer_id', 
    'transaction_date',
    'amount',
    'country', 
    'city', 
    'currency', 
    'mcc', 
    'pos_adress_lat', 
    'pos_adress_lon', 
    'atm_address_lat', 
    'atm_address_lon',
    'home_add_lat',
    'home_add_lon',
    'work_add_lat',
    'work_add_lon', 
    'terminal_id'
]
usecols_test = [
    'customer_id',
    'transaction_date',
    'amount',
    'country', 
    'city', 
    'currency', 
    'mcc', 
    'pos_address_lat', 
    'pos_address_lon', 
    'atm_address_lat', 
    'atm_address_lon',
    'terminal_id'
]

In [5]:
%%time

train_df = pd.read_csv(r"../../data/train_set-Raiffaizen.csv", dtype=dtypes, usecols=usecols_train)  # low_memory=False
train_df.rename(columns={'pos_adress_lat': 'pos_address_lat', 'pos_adress_lon': 'pos_address_lon'}, inplace=True)

test_df = pd.read_csv(r"../../data/test_set-Raiffaizen.csv", dtype=dtypes, usecols=usecols_test)  # low_memory=False
submission = pd.DataFrame(test_df.customer_id.unique(), columns = ['_ID_'])

# соединяем test/train в одном DataFrame
train_df['is_train'] = np.int32(1)
test_df['is_train'] = np.int32(0)
df = pd.concat([train_df, test_df])

df.sample(5)

del train_df, test_df

CPU times: user 8.08 s, sys: 550 ms, total: 8.63 s
Wall time: 8.91 s


In [6]:
%%time

df.currency = df.currency.fillna(-1).astype(np.int32)
df.mcc = df.mcc.apply(lambda x: int(x.replace(',', ''))).astype(np.int32)

# удаляем транзакции без даты
df.drop(df[df.transaction_date.isnull()].index, axis=0, inplace=True)
df.transaction_date = df.transaction_date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df['weekday'] = df.transaction_date.dt.weekday.astype(np.int32)

CPU times: user 28.1 s, sys: 320 ms, total: 28.4 s
Wall time: 28.5 s


In [7]:
df.shape

(2490114, 18)

#### Очистка названий городов (попытка убрать лишние классы).

In [8]:
pattern = re.compile("[^ a-zA-Z]")
def filter_city(city: str, min_len: int=2) -> list:
    t = pattern.sub(r"", city.lower())
    return " ".join([c for c in t.strip().split() if len(c) > min_len])

In [9]:
df.city = df.city.apply(lambda x: str.lower(x) if x is not np.NaN else "")
df.city = df.city.apply(filter_city)
df.city = df.city.factorize()[0].astype(np.int32)

In [10]:
df.country = df.country.apply(lambda x: str.lower(x) if x is not np.NaN else "")
df.country = df.country.apply(filter_city)
df.country = df.country.factorize()[0].astype(np.int32)

#### Объединение координат адресов pos- и atm- терминалов.

In [11]:
%%time

df['is_atm'] = (~df.atm_address_lat.isnull()).astype(np.int32)
df['is_pos'] = (~df.pos_address_lat.isnull()).astype(np.int32)

df['address_lat'] = df.atm_address_lat.fillna(0) + df.pos_address_lat.fillna(0)
df['address_lon'] = df.atm_address_lon.fillna(0) + df.pos_address_lon.fillna(0)

df.drop(['atm_address_lat','atm_address_lon','pos_address_lat','pos_address_lon'], axis=1, inplace=True)

# удалим транзакции без адреса
df.drop(df[((df.address_lon == 0) & (df.address_lon == 0))].index, axis=0, inplace=True)

CPU times: user 1.09 s, sys: 140 ms, total: 1.23 s
Wall time: 1.23 s


#### Создание бинарных признаков is_home и is_work, категоризация признака "адрес".

In [12]:
def custom_metrics(lat: np.ndarray, lon: np.ndarray, eps: float=0.02) -> np.ndarray:
    return (np.sqrt((lat**2) + (lon**2)) <= eps).astype(np.int32)

In [13]:
%%time

lat_h = df.home_add_lat - df.address_lat
lon_h = df.home_add_lon - df.address_lon
df['is_home'] = custom_metrics(lat_h, lon_h)
df['has_home'] = (~df.home_add_lon.isnull()).astype(np.int32)

lat_w = df.work_add_lat - df.address_lat
lon_w = df.work_add_lon - df.address_lon
df['is_work'] = custom_metrics(lat_w, lon_w)
df['has_work'] = (~df.work_add_lon.isnull()).astype(np.int32)

df.drop(['work_add_lat','work_add_lon','home_add_lat','home_add_lon'], axis=1, inplace=True)

del lat_h, lon_h, lat_w, lon_w

CPU times: user 350 ms, sys: 80 ms, total: 430 ms
Wall time: 436 ms


#### Усреднение местоположения для банкоматов.

In [14]:
%%time

# средние координаты терминалов
df = df.merge(df.groupby('terminal_id').address_lat.mean().reset_index(name='terminal_lat'), how='left')
df = df.merge(df.groupby('terminal_id').address_lon.mean().reset_index(name='terminal_lon'), how='left')

df.drop(['address_lat','address_lon'], axis=1, inplace=True)

CPU times: user 4.63 s, sys: 150 ms, total: 4.78 s
Wall time: 4.78 s


In [15]:
addr_lat = df.terminal_lat.apply(lambda x: "%.02f" % x)
addr_lon = df.terminal_lon.apply(lambda x: "%.02f" % x)
df['address'] = addr_lat + ';' + addr_lon
df.address = df.address.factorize()[0].astype(np.int32)
del addr_lat, addr_lon

In [16]:
df.head()

,amount,city,country,currency,customer_id,is_train,mcc,terminal_id,transaction_date,weekday,is_atm,is_pos,is_home,has_home,is_work,has_work,terminal_lat,terminal_lon,address
0,2.884034,0,0,643,0dc0137d280a2a82d2dc89282450ff1b,1,5261,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,5,0,1,0,1,1,1,59.844074,30.179153,0
1,2.775633,0,0,643,0dc0137d280a2a82d2dc89282450ff1b,1,5261,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,4,0,1,0,1,1,1,59.844074,30.179153,0
2,3.708368,0,0,643,0dc0137d280a2a82d2dc89282450ff1b,1,5992,df06c1fcd3718a514535ae822785f716,2017-10-03,1,0,1,1,1,0,1,59.858200,30.229023,1
3,2.787498,0,0,643,0dc0137d280a2a82d2dc89282450ff1b,1,5261,6c5e5793ebc984fb72875feffff62854,2017-09-09,5,0,1,0,1,1,1,59.844074,30.179153,0
4,2.892510,0,0,643,0dc0137d280a2a82d2dc89282450ff1b,1,5261,0576445d74e374c92c0902e612fca356,2017-07-06,3,0,1,0,1,1,1,59.844074,30.179153,0


In [17]:
%%time

# количество транзакций каждого клиента
df = df.merge(df.groupby('customer_id').amount.count().reset_index(name='customer_trans'), how='left')
df.customer_trans = df.customer_trans.astype(np.int32)

df = df.merge(df.groupby(['customer_id','address']).amount.count().reset_index(name='customer_trans_addr'), how='left')
df.customer_trans_addr = df.customer_trans_addr.astype(np.int32)

# какая часть транзакций клиента приходится на данный адрес
df['ratio1'] = df.customer_trans_addr / df.customer_trans

# средняя сумма клиента, снятая в банкомате
df = df.merge(df.groupby(['customer_id', 'address']).amount.mean().reset_index(name='pos_mean'), how='left')
df = df.merge(df.groupby(['customer_id', 'address']).amount.sum().reset_index(name='pos_amount'), how='left')

CPU times: user 4.87 s, sys: 210 ms, total: 5.08 s
Wall time: 5.09 s


#### Определение района.

In [18]:
%%time

coordinates = tuple(np.hstack((
    df.terminal_lat.values.reshape(-1, 1), df.terminal_lon.values.reshape(-1, 1)
)))
coordinates = tuple(map(tuple, coordinates))
disctricts = np.array(list(map(
    lambda x: (x['name']), reverse_geocoder.search(coordinates, verbose=False)
)))
df['district'] = df.city.apply(str) + '_' + disctricts
df.district = df.district.factorize()[0].astype(np.int32)

CPU times: user 8.77 s, sys: 780 ms, total: 9.55 s
Wall time: 11.6 s


In [19]:
df.head()

,amount,city,country,currency,customer_id,is_train,mcc,terminal_id,transaction_date,weekday,...,has_work,terminal_lat,terminal_lon,address,customer_trans,customer_trans_addr,ratio1,pos_mean,pos_amount,district
0,2.884034,0,0,643,0dc0137d280a2a82d2dc89282450ff1b,1,5261,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,5,...,1,59.844074,30.179153,0,39,17,0.435897,3.298660,56.077213,0
1,2.775633,0,0,643,0dc0137d280a2a82d2dc89282450ff1b,1,5261,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,4,...,1,59.844074,30.179153,0,39,17,0.435897,3.298660,56.077213,0
2,3.708368,0,0,643,0dc0137d280a2a82d2dc89282450ff1b,1,5992,df06c1fcd3718a514535ae822785f716,2017-10-03,1,...,1,59.858200,30.229023,1,39,5,0.128205,3.679532,18.397659,1
3,2.787498,0,0,643,0dc0137d280a2a82d2dc89282450ff1b,1,5261,6c5e5793ebc984fb72875feffff62854,2017-09-09,5,...,1,59.844074,30.179153,0,39,17,0.435897,3.298660,56.077213,0
4,2.892510,0,0,643,0dc0137d280a2a82d2dc89282450ff1b,1,5261,0576445d74e374c92c0902e612fca356,2017-07-06,3,...,1,59.844074,30.179153,0,39,17,0.435897,3.298660,56.077213,0


#### Вспомогательные функции для оценки точности классификатора

In [20]:
def _best(x):
    ret = None
    for col in ys:
        pred = ('pred:%s' % col)
        if pred in x:
            i = (x[pred].idxmax())
            cols = [pred,'terminal_lat','terminal_lon']
            if col in x:
                cols.append(col)
            tmp = x.loc[i,cols]
            tmp.rename({
                'terminal_lat':'%s:add_lat' % col,
                'terminal_lon':'%s:add_lon' % col,
            }, inplace = True)
            if ret is None:
                ret = tmp
            else:
                ret = pd.concat([ret, tmp])
    return ret

def predict_proba(df, ys=['is_home', 'is_work']):
    for col in ys:
        pred = ('pred:%s' % col)
        df[pred] = model[col].predict_proba(df[xs])[:,1]
    return df.groupby('customer_id').apply(_best).reset_index()

def score(df, ys=['is_home', 'is_work']):
    df_ret = predict_proba(df, ys)
    mean = 0.0
    for col in ys:
        col_mean = df_ret[col].mean()
        mean += col_mean
    if len(ys) == 2:
        mean = mean / len(ys)
    return mean

#### Обучение

In [65]:
# фичи, на которых идет обучение
xs = [
    'amount',
#     'currency',
    'city',
#     'country',
    'mcc',
    'is_atm',
    'ratio1',
    'district', 
    'address', 
    'pos_amount',
    'pos_mean',
#     'customer_trans_addr', 
    'weekday'
]
ys = ['is_home', 'is_work']

# модели
model0 = {
    'is_home': xgb.XGBClassifier(max_depth=4, n_estimators=200, nthread=3, random_state=RANDOM_SEED),
    'is_work': xgb.XGBClassifier(n_estimators=150, nthread=3, random_state=RANDOM_SEED)
}

model = {}

In [66]:
def fit_models(x_idx: list, verbose: int=25, models: dict={}) -> dict:
    # последовательно обучаем два классификатора
    for col in ['is_home', 'is_work']:
        #выберем для обучение транзакции только тех клиентов из train, у которых хоть в одной транзакции указано место работы/жительства
        cust_train = df[df.is_train == 1].groupby('customer_id')[col.replace('is_','has_')].max()
        cust_train = cust_train[cust_train > 0].index

        #разобъем train на train/valid для валидации
        cust_train, cust_valid = train_test_split(cust_train, test_size=0.2, 
                                                  shuffle=True, random_state=RANDOM_SEED)

        train = pd.DataFrame(cust_train, columns=['customer_id']).merge(df, how='left')
        valid = pd.DataFrame(cust_valid, columns=['customer_id']).merge(df, how='left')

        print ("Training:", col)
        clf = sklearn.base.clone(model0[col])
        clf.fit(train[x_idx], train[col], eval_metric='logloss', 
                eval_set=[(train[x_idx], train[col]), (valid[x_idx], valid[col])], verbose=verbose)
        model[col] = clf
        print ("Train accuracy:", score(train, ys=[col]))
        print ("Test accuracy:", score(valid, ys=[col]))
        print ()
    return model

In [67]:
%%time

model = fit_models(xs, 10, model)

Training: is_home
[0]	validation_0-logloss:0.659042	validation_1-logloss:0.658413
[10]	validation_0-logloss:0.514481	validation_1-logloss:0.511787
[20]	validation_0-logloss:0.480596	validation_1-logloss:0.479243
[30]	validation_0-logloss:0.466995	validation_1-logloss:0.467237
[40]	validation_0-logloss:0.459382	validation_1-logloss:0.46141
[50]	validation_0-logloss:0.453781	validation_1-logloss:0.458654
[60]	validation_0-logloss:0.450092	validation_1-logloss:0.456802
[70]	validation_0-logloss:0.446665	validation_1-logloss:0.455987
[80]	validation_0-logloss:0.443862	validation_1-logloss:0.455411
[90]	validation_0-logloss:0.441109	validation_1-logloss:0.455621
[100]	validation_0-logloss:0.4383	validation_1-logloss:0.45487
[110]	validation_0-logloss:0.435111	validation_1-logloss:0.454176
[120]	validation_0-logloss:0.432053	validation_1-logloss:0.453705
[130]	validation_0-logloss:0.429205	validation_1-logloss:0.45246
[140]	validation_0-logloss:0.427078	validation_1-logloss:0.451731
[150]	va

In [68]:
pd.DataFrame(model['is_home'].feature_importances_, index=xs).T

,amount,city,mcc,is_atm,ratio1,district,address,pos_amount,pos_mean,weekday
0,0.013776,0.074885,0.095726,0.007065,0.152949,0.156835,0.146945,0.20982,0.132462,0.009537


In [69]:
pd.DataFrame(model['is_work'].feature_importances_, index=xs).T

,amount,city,mcc,is_atm,ratio1,district,address,pos_amount,pos_mean,weekday
0,0.013645,0.078947,0.059454,0.016569,0.149123,0.192008,0.146199,0.19883,0.116959,0.028265


```python
%%time

main_xs = ['city', 'mcc', 'ratio1', 'district', 'address', 'weekday']
good_xs = ['customer_trans_addr', 'pos_mean', 'pos_amount']
other_xs = ['amount', 'is_atm']
XS = [
    main_xs + good_xs + other_xs,  # full
    main_xs + good_xs,
    main_xs,
    main_xs + other_xs,
    main_xs + good_xs[:-1],
    main_xs + good_xs[1:]
]

models = []
feature_importances = []

for xs in XS:
    models.append(fit_models(xs, verbose=50))
```

#### Предсказание.

In [70]:
%%time

cust_test = df[df.is_train == 0].customer_id.unique()
test = pd.DataFrame(cust_test, columns=['customer_id']).merge(df, how='left')
test = predict_proba(test)
test.rename(inplace=True, columns={
    'customer_id': '_ID_',
    'is_home:add_lat': '_HOME_LAT_',
    'is_home:add_lon': '_HOME_LON_',
    'is_work:add_lat': '_WORK_LAT_',
    'is_work:add_lon': '_WORK_LON_'
})
test = test[['_ID_', '_WORK_LAT_', '_WORK_LON_', '_HOME_LAT_', '_HOME_LON_']]

CPU times: user 49.3 s, sys: 130 ms, total: 49.4 s
Wall time: 49.5 s


In [72]:
# Заполняем пропуски
submission = submission.merge(test, how='left').fillna(0)

# Пишем файл submission
submission.to_csv('submission_final.csv', index=False)